# LORA Fine-tuning

In [3]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np 

In [ ]:
# load dataset
dataset = load_dataset('jackhhao/jailbreak-classification')
dataset = dataset.rename_column("prompt", "text")
dataset = dataset.rename_column("type", "label")
dataset = dataset.cast_column("label", datasets.ClassLabel(names=["benign", "jailbreak"]))

dataset

### model

In [ ]:
model_checkpoint = "distilbert-base-uncased"
model_output = "distilbert-base-uncased-lora-text-classification-jackhao-jailbreak"

# define label maps
id2label = {0: "Good", 1: "Jailbreak"}
label2id = {"Good": 0, "Jailbreak": 1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id, trust_remote_code=True)

In [ ]:
# display architecture
model

### preprocess data

In [7]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
# create tokenize function
def tokenize(batch):

    #tokenize and truncate text
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    print(batch['label'])
    return tokens


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset


In [9]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

In [10]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

### Apply untrained model to text

In [ ]:
# define list of examples
text_list = ["jailbreak [keyword] [TARGETLANGUAGE] [PROMPT]"]
print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    #print(inputs)
    # compute logits
    logits = model(inputs).logits
    #print(logits)
    # convert logits to label
    predictions = torch.argmax(logits)
    #print(predictions)

    print(id2label[predictions.tolist()])

### Train model

In [12]:

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules=["q_lin", "k_lin", "v_lin"])

In [ ]:
peft_config

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [15]:
# hyperparameters
lr = 1e-3
batch_size = 2
num_epochs = 5

In [16]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_output,
    learning_rate=lr,
    auto_find_batch_size=True,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [17]:
from sklearn.metrics import f1_score, roc_auc_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    auprc = roc_auc_score(labels, preds, average='weighted')
    return {
    'f1': f1,
    'auprc': auprc
    }

In [18]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics
)

In [ ]:
# train model
trainer.train()

### Generate prediction

In [ ]:
model.to('cpu') # moving to mps for Mac (can alternatively do 'cpu')

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu") # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(id2label[predictions.tolist()[0]])

### Optional: push model to hub

In [65]:
# option 1: notebook login
# from huggingface_hub import notebook_login
# notebook_login() # ensure token gives write access

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [66]:
hf_name = 'cyrp' # your hf username or org name
model_id = hf_name + "/" + model_output # you can name the model whatever you want

In [ ]:
model.push_to_hub(model_id) # save model

In [ ]:
trainer.push_to_hub(model_output) # save trainer

### Optional: load peft model

In [69]:
# how to load peft model from hub for inference
# config = PeftConfig.from_pretrained(model_id)
# inference_model = AutoModelForSequenceClassification.from_pretrained(
#     config.base_model_name_or_path, num_labels=2, id2label=id2label, label2id=label2id
# )
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = PeftModel.from_pretrained(inference_model, model_id)